In [1]:
from keras import *
from keras.layers import Dense,Conv2D,Conv1D,MaxPooling2D, UpSampling2D,Embedding
from keras.models import Model,Sequential
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import keras_preprocessing 
import os,scipy
import tensorflow as tf
from keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# from keras.backend.tensorflow_backend import set_session
# config = tf.ConfigProto(device_count={'gpu':0})
# config.gpu_options.per_process_gpu_memory_fraction = 0.9  # 为显卡利用率设置阈值
# set_session(tf.Session(config=config))


num_sample = 12000
cut_point = int(0.7 * num_sample)
bit  = 1
bit_k = 8
category = pow(2,bit_k)
r_dim = 1

epoch = 100
batch_size = 25

##  这里是numpy下进行one-hot编码的函数，
##  不同于pandas下的直接调用get_dummies,pandas中设计 Dataframe 的转化
def convert_to_one_hot(y, C):
    return np.eye(C)[y.reshape(-1)]

# source = ['0000','0001','0010','0011',
#           '0100','0101','0110','0111',
#           '1000','1001','1010','1011',
#           '1100','1101','1110','1111'] 

I = np.array(np.zeros([num_sample,bit]),dtype=int)
Q = np.array(np.zeros([num_sample,bit]),dtype=int)

I_k = np.array(np.zeros([num_sample,bit_k]),dtype=int)
Q_k = np.array(np.zeros([num_sample,bit_k]),dtype=int)

for i in range(num_sample):
    
    #产生二进制对应的整数
    row_I = np.random.randint(16, size=[r_dim,bit],dtype=int)
    row_Q = np.random.randint(16, size=[r_dim,bit],dtype=int)
    I[i] = row_I
    Q[i] = row_Q
    
    # 产生4位二进制数
    row_I_k = np.random.randint(2, size=[r_dim,bit_k],dtype=int)
    row_Q_k = np.random.randint(2, size=[r_dim,bit_k],dtype=int)
    I_k[i] = row_I_k
    Q_k[i] = row_I_k

# 构建I路数据集，标签集
I_dataset_data = I_k.reshape([num_sample,bit_k])
I_dataset_label = convert_to_one_hot(I,category)
I_dataset = np.append(I_k,I_dataset_label,axis=1)

# 构建Q路数据集，标签集
Q_dataset_data = Q_k.reshape([num_sample,bit_k])
Q_dataset_label = convert_to_one_hot(Q,category)
Q_dataset = np.append(Q_k,Q_dataset_label,axis=1)

# 构建IQ数据集，标签集
# IQ_dataset_data为IQ数据,共8bit,各4bit
# IQ_dataset_label为IQ数据标签，one-hot形式，各16bit,共32bit
IQ_dataset_data = np.append(I_dataset_data,Q_dataset_data,axis=1)
IQ_dataset_label = np.append(I_dataset_label,Q_dataset_label,axis=1)
IQ_dataset = np.append(IQ_dataset_data,IQ_dataset_label,axis=1)
# print(IQ_dataset.shape)

#############################################################
# 这里最终将一个二维数组化为了12000个2*4*1的三维数组，即12000个切片
# 后面可以利用Conv2D进行卷积处理 
# IQ_data = np.reshape(IQ_dataset_data, (-1,2,bit_k,1))
# print(IQ_dataset_label.shape) 
#############################################################
X_train, X_test, y_train, y_test = train_test_split(IQ_dataset_data, 
        IQ_dataset_label,test_size=0.3, random_state=0) # 为了看模型在没有见过数据集上的表现，随机拿出数据集中30%的部分做测试
X_train = X_train.reshape(-1,2,bit_k,1)
X_test = X_test.reshape(-1,2,bit_k,1)
# y_train = X_test.reshape(-1,2,bit_k,1)

# print(X_train.shape,X_test.shape)
# X_train.reshape(-1,)
############ 根据文章自己写的编码器结构 ###############################
input_shape = (2,bit_k,1)
input_data = Input(input_shape)
# x = Conv2D(32,(3,3),activation="linear",padding='same')(input_data)
# x = Dense(44,activation='hard_sigmoid')(x)
# x = Dense(176,activation='hard_sigmoid')(x)
# x = Conv2D(48,(3,3),activation='linear',padding='same')(x)

x = Conv2D(16, (2,2), activation='relu', padding='same')(input_data)
x = MaxPooling2D((2,2), padding='same')(x)
x = Conv2D(8,(2,2), activation='relu', padding='same')(x)
x = MaxPooling2D((2,2), padding='same')(x)
x = Conv2D(8, (2,32), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2,2), padding='same')(x)

# 解码器部分
x = Conv2D(8, (2,2), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 4))(x)
x = Conv2D(16, (2,4), activation='relu', padding='same')(x) 
x = UpSampling2D((1, 2))(x)
decoded = Conv2D(1, (2, 2), activation='sigmoid', padding='same')(x)

autoencoder = Model(inputs=input_data, outputs=decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.summary()

## 得到编码层的输出
# encoder_model = Model(inputs=autoencoder.input, 
#   outputs=autoencoder.get_layer('encoded').output)
autoencoder.fit(X_train,X_train, epochs=epoch, batch_size=batch_size,    
                shuffle=True, validation_data=(X_test,X_test),  
    callbacks=[TensorBoard(log_dir='./tmp/log')])
decoded_data = autoencoder.predict(X_test)
# print(decoded_data)


Using TensorFlow backend.
C:\ProgramData\Anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\utils\fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2, 8, 1)           0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 2, 8, 16)          80        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 4, 16)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 4, 8)           520       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 2, 8)           0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 1, 2, 8)           4104      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 1, 1, 8)           0         
__________

Epoch 60/100
8400/8400 [==============================] - 1s 171us/step - loss: 0.0419 - val_loss: 0.0486
Epoch 61/100
8400/8400 [==============================] - 1s 175us/step - loss: 0.0410 - val_loss: 0.0402
Epoch 62/100
8400/8400 [==============================] - 2s 191us/step - loss: 0.0487 - val_loss: 0.0419
Epoch 63/100
8400/8400 [==============================] - 2s 185us/step - loss: 0.0385 - val_loss: 0.0386
Epoch 64/100
8400/8400 [==============================] - 2s 213us/step - loss: 0.0368 - val_loss: 0.0411
Epoch 65/100
8400/8400 [==============================] - 1s 173us/step - loss: 0.0367 - val_loss: 0.0350
Epoch 66/100
8400/8400 [==============================] - 1s 177us/step - loss: 0.0359 - val_loss: 0.0331
Epoch 67/100
8400/8400 [==============================] - 1s 174us/step - loss: 0.0348 - val_loss: 0.0358
Epoch 68/100
8400/8400 [==============================] - 2s 179us/step - loss: 0.0351 - val_loss: 0.0311
Epoch 69/100
8400/8400 [======================

In [2]:
print(decoded_data.shape)
decoded_data_bk = decoded_data
# from sklearn.manifold import TSNE
# TSNE

(3600, 2, 8, 1)
